In [44]:
import numpy as np
import pandas as pd
import mgrs

In [45]:
def missingByCol(df): #count missing values by column
    for col in df.columns:
        if df[col].isna().sum() != 0:
            print(col, df[col].isna().sum())

def getCalTbl(start='2000-01-01', end='2015-12-31'):
    df = pd.DataFrame({'date': pd.date_range(start, end)})
    return df

Import Weather Data

In [46]:
weather = pd.read_csv('2000to2015_Weather_v2.csv')
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2249940 entries, 0 to 2249939
Data columns (total 14 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Unnamed: 0                     int64  
 1   lat                            float64
 2   lon                            float64
 3   t_mean_2m_24h:F                float64
 4   t_min_2m_24h:F                 float64
 5   t_max_2m_24h:F                 float64
 6   precip_24h:mm                  float64
 7   sunshine_duration_24h:min      float64
 8   drought_index:idx              float64
 9   soil_moisture_index_-15cm:idx  float64
 10  soil_type:idx                  float64
 11  wind_speed_2m:mph              float64
 12  elevation:m                    float64
 13  date                           object 
dtypes: float64(12), int64(1), object(1)
memory usage: 240.3+ MB


In [47]:
#get calendar table
calendar = getCalTbl()
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5844 entries, 0 to 5843
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5844 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 45.8 KB


Reattach MGRS 10km blocks to Weather Data

In [48]:
#Import mgrs locations
mgrs_final = pd.read_csv('https://raw.githubusercontent.com/jbalson0808/fires/main/mgrs_final.csv') #set location of mgrs_final
mgrs_final = mgrs_final[['mgrs_10km','lat_10km','lon_10km']]
mgrs_final.info() 

mgrs_10km = []
for i in mgrs_final['mgrs_10km'].tolist():
    # print(i)
    for j in range(len(calendar)):
        # print(i,j)
        mgrs_10km.append(i)
len(mgrs_10km)
weather['mgrs_10km'] = mgrs_10km
weather.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   mgrs_10km  385 non-null    object 
 1   lat_10km   385 non-null    float64
 2   lon_10km   385 non-null    float64
dtypes: float64(2), object(1)
memory usage: 9.1+ KB


,Unnamed: 0,lat,lon,t_mean_2m_24h:F,t_min_2m_24h:F,t_max_2m_24h:F,precip_24h:mm,sunshine_duration_24h:min,drought_index:idx,soil_moisture_index_-15cm:idx,soil_type:idx,wind_speed_2m:mph,elevation:m,date,mgrs_10km
2249935,2249935,33.167642,-117.536254,54.2,48.3,61.9,0.00,598.0,-1.0,NaN,NaN,4.5,0.0,2015-12-27 00:00:00+00:00,11SMS57
2249936,2249936,33.167642,-117.536254,53.6,49.7,57.2,0.08,212.6,-1.0,NaN,NaN,3.1,0.0,2015-12-28 00:00:00+00:00,11SMS57
2249937,2249937,33.167642,-117.536254,52.6,48.1,56.8,0.07,557.9,-1.0,NaN,NaN,1.8,0.0,2015-12-29 00:00:00+00:00,11SMS57
2249938,2249938,33.167642,-117.536254,53.9,49.7,57.7,0.00,568.3,-2.0,NaN,NaN,7.2,0.0,2015-12-30 00:00:00+00:00,11SMS57
2249939,2249939,33.167642,-117.536254,54.9,50.3,60.2,0.00,599.0,-2.0,NaN,NaN,7.9,0.0,2015-12-31 00:00:00+00:00,11SMS57


Impute Missing Values for Soil Type

In [49]:
missingByCol(weather)

soil_moisture_index_-15cm:idx 138117
soil_type:idx 136651


In [50]:
weather['soil_type:idx'] = weather['soil_type:idx'].fillna(4)
missingByCol(weather)

soil_moisture_index_-15cm:idx 138117


In [51]:
weather['soil_moisture_index_-15cm:idx'] = weather['soil_moisture_index_-15cm:idx'].fillna(0.068733)
missingByCol(weather)

Preprocessing

In [52]:
weather = weather[['date','mgrs_10km','lat','lon','t_mean_2m_24h:F','t_min_2m_24h:F','t_max_2m_24h:F','precip_24h:mm','sunshine_duration_24h:min','drought_index:idx','wind_speed_2m:mph','elevation:m','soil_type:idx','soil_moisture_index_-15cm:idx']]
weather = weather.rename(columns={'t_mean_2m_24h:F':'tavg','t_min_2m_24h:F':'tmin','t_max_2m_24h:F':'tmax','precip_24h:mm':'prcp','sunshine_duration_24h:min':'sun','drought_index:idx':'drought_index','wind_speed_2m:mph':'wind_speed','elevation:m':'elevation','soil_type:idx':'soil_type','soil_moisture_index_-15cm:idx':'soil_moisture'
})
weather.date = pd.to_datetime(pd.to_datetime(weather['date']).dt.date)
# weather.date = weather.date.dt.date
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2249940 entries, 0 to 2249939
Data columns (total 14 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date           datetime64[ns]
 1   mgrs_10km      object        
 2   lat            float64       
 3   lon            float64       
 4   tavg           float64       
 5   tmin           float64       
 6   tmax           float64       
 7   prcp           float64       
 8   sun            float64       
 9   drought_index  float64       
 10  wind_speed     float64       
 11  elevation      float64       
 12  soil_type      float64       
 13  soil_moisture  float64       
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 240.3+ MB


In [53]:
len(weather.mgrs_10km.unique())

385

In [54]:
len(weather.date.unique())

5844

In [55]:
# rolling averages for prcp, tmax, tmin for 3, 7, and 10 days

#prcp
weather['prcp_rolling_3day'] = weather.groupby(['lat','lon'])['prcp'].transform(lambda x: x.rolling(3).mean())
weather['prcp_rolling_3day'] = round(weather['prcp_rolling_3day'],6)

weather['prcp_rolling_7day'] = weather.groupby(['lat','lon'])['prcp'].transform(lambda x: x.rolling(7).mean())
weather['prcp_rolling_7day'] = round(weather['prcp_rolling_7day'],6)

weather['prcp_rolling_14day'] = weather.groupby(['lat','lon'])['prcp'].transform(lambda x: x.rolling(14).mean())
weather['prcp_rolling_14day'] = round(weather['prcp_rolling_14day'],6)

#tavg
weather['tavg_rolling_3day'] = weather.groupby(['lat','lon'])['tavg'].transform(lambda x: x.rolling(3).mean())
weather['tavg_rolling_3day'] = round(weather['tavg_rolling_3day'],6)

weather['tavg_rolling_7day'] = weather.groupby(['lat','lon'])['tavg'].transform(lambda x: x.rolling(7).mean())
weather['tavg_rolling_7day'] = round(weather['tavg_rolling_7day'],6)

weather['tavg_rolling_14day'] = weather.groupby(['lat','lon'])['tavg'].transform(lambda x: x.rolling(14).mean())
weather['tavg_rolling_14day'] = round(weather['tavg_rolling_14day'],6)

#tmin
weather['tmin_rolling_3day'] = weather.groupby(['lat','lon'])['tmin'].transform(lambda x: x.rolling(3).mean())
weather['tmin_rolling_3day'] = round(weather['tmin_rolling_3day'],6)

weather['tmin_rolling_7day'] = weather.groupby(['lat','lon'])['tmin'].transform(lambda x: x.rolling(7).mean())
weather['tmin_rolling_7day'] = round(weather['tmin_rolling_7day'],6)

weather['tmin_rolling_14day'] = weather.groupby(['lat','lon'])['tmin'].transform(lambda x: x.rolling(14).mean())
weather['tmin_rolling_14day'] = round(weather['tmin_rolling_14day'],6)

#tmax
weather['tmax_rolling_3day'] = weather.groupby(['lat','lon'])['tmax'].transform(lambda x: x.rolling(3).mean())
weather['tmax_rolling_3day'] = round(weather['tmax_rolling_3day'],6)

weather['tmax_rolling_7day'] = weather.groupby(['lat','lon'])['tmax'].transform(lambda x: x.rolling(7).mean())
weather['tmax_rolling_7day'] = round(weather['tmax_rolling_7day'],6)

weather['tmax_rolling_14day'] = weather.groupby(['lat','lon'])['tmax'].transform(lambda x: x.rolling(14).mean())
weather['tmax_rolling_14day'] = round(weather['tmax_rolling_14day'],6)

#sun
weather['sun_rolling_3day'] = weather.groupby(['lat','lon'])['sun'].transform(lambda x: x.rolling(3).mean())
weather['sun_rolling_3day'] = round(weather['sun_rolling_3day'],6)

weather['sun_rolling_7day'] = weather.groupby(['lat','lon'])['sun'].transform(lambda x: x.rolling(7).mean())
weather['sun_rolling_7day'] = round(weather['sun_rolling_7day'],6)

weather['sun_rolling_14day'] = weather.groupby(['lat','lon'])['sun'].transform(lambda x: x.rolling(14).mean())
weather['sun_rolling_14day'] = round(weather['sun_rolling_14day'],6)

#windspeed
weather['wind_speed_3day'] = weather.groupby(['lat','lon'])['wind_speed'].transform(lambda x: x.rolling(3).mean())
weather['wind_speed_3day'] = round(weather['wind_speed_3day'],6)

weather['wind_speed_7day'] = weather.groupby(['lat','lon'])['wind_speed'].transform(lambda x: x.rolling(7).mean())
weather['wind_speed_7day'] = round(weather['wind_speed_7day'],6)

weather['wind_speed_14day'] = weather.groupby(['lat','lon'])['wind_speed'].transform(lambda x: x.rolling(14).mean())
weather['wind_speed_14day'] = round(weather['wind_speed_14day'],6)

#soil moisture
weather['soil_moisture_3day'] = weather.groupby(['lat','lon'])['soil_moisture'].transform(lambda x: x.rolling(3).mean())
weather['soil_moisture_3day'] = round(weather['soil_moisture_3day'],6)

weather['soil_moisture_7day'] = weather.groupby(['lat','lon'])['soil_moisture'].transform(lambda x: x.rolling(7).mean())
weather['soil_moisture_7day'] = round(weather['soil_moisture_7day'],6)

weather['soil_moisture_14day'] = weather.groupby(['lat','lon'])['soil_moisture'].transform(lambda x: x.rolling(14).mean())
weather['soil_moisture_14day'] = round(weather['soil_moisture_14day'],6)

In [56]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2249940 entries, 0 to 2249939
Data columns (total 35 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date                 datetime64[ns]
 1   mgrs_10km            object        
 2   lat                  float64       
 3   lon                  float64       
 4   tavg                 float64       
 5   tmin                 float64       
 6   tmax                 float64       
 7   prcp                 float64       
 8   sun                  float64       
 9   drought_index        float64       
 10  wind_speed           float64       
 11  elevation            float64       
 12  soil_type            float64       
 13  soil_moisture        float64       
 14  prcp_rolling_3day    float64       
 15  prcp_rolling_7day    float64       
 16  prcp_rolling_14day   float64       
 17  tavg_rolling_3day    float64       
 18  tavg_rolling_7day    float64       
 19  tavg_rolling_14day   

In [57]:
missingByCol(weather) # these will be removed when the fire data is merged

prcp_rolling_3day 770
prcp_rolling_7day 2310
prcp_rolling_14day 5005
tavg_rolling_3day 770
tavg_rolling_7day 2310
tavg_rolling_14day 5005
tmin_rolling_3day 770
tmin_rolling_7day 2310
tmin_rolling_14day 5005
tmax_rolling_3day 770
tmax_rolling_7day 2310
tmax_rolling_14day 5005
sun_rolling_3day 770
sun_rolling_7day 2310
sun_rolling_14day 5005
wind_speed_3day 770
wind_speed_7day 2310
wind_speed_14day 5005
soil_moisture_3day 770
soil_moisture_7day 2310
soil_moisture_14day 5005


Get Population Density Data

In [58]:
pop_density_mgrs = pd.read_csv('pop_density_mgrs.csv')
pop_density_mgrs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mgrs_10km            385 non-null    object 
 1   Density Per Sq Mile  385 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.1+ KB


In [59]:
weather = weather.merge(pop_density_mgrs,on='mgrs_10km',how='left')
weather = weather.rename(columns={'Density Per Sq Mile':'pop_density_sqmile'})
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2249940 entries, 0 to 2249939
Data columns (total 36 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date                 datetime64[ns]
 1   mgrs_10km            object        
 2   lat                  float64       
 3   lon                  float64       
 4   tavg                 float64       
 5   tmin                 float64       
 6   tmax                 float64       
 7   prcp                 float64       
 8   sun                  float64       
 9   drought_index        float64       
 10  wind_speed           float64       
 11  elevation            float64       
 12  soil_type            float64       
 13  soil_moisture        float64       
 14  prcp_rolling_3day    float64       
 15  prcp_rolling_7day    float64       
 16  prcp_rolling_14day   float64       
 17  tavg_rolling_3day    float64       
 18  tavg_rolling_7day    float64       
 19  tavg_rolling_14day   

In [60]:
weather.to_csv('weather_clean.csv',index=False)